# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,3.43,87,100,7.72,CA,1706545203
1,1,talnakh,69.4865,88.3972,-14.33,90,100,3.93,RU,1706545235
2,2,cabo san lucas,22.8909,-109.9124,25.98,51,40,1.03,MX,1706545177
3,3,sarkand,45.4100,79.9186,-7.69,84,100,2.00,KZ,1706545236
4,4,invercargill,-46.4000,168.3500,13.34,88,90,1.80,NZ,1706544966


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Humidity"]
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_Weather_df = city_data_df[city_data_df["Max Temp"]>25] #25 celcius is 65 Farenheit
ideal_Weather_df = city_data_df[city_data_df["Humidity"]<40]
# Drop any rows with null values
ideal_Weather_df = ideal_Weather_df.dropna()

# Display sample data
ideal_Weather_df.count()

City_ID       53
City          53
Lat           53
Lng           53
Max Temp      53
Humidity      53
Cloudiness    53
Wind Speed    53
Country       53
Date          53
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_Weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,51,kandi,11.1342,2.9386,34.81,10,50,2.12,BJ,1706545247,NaN
53,53,bam,29.1060,58.3570,13.37,28,100,2.20,IR,1706545247,NaN
68,68,panguma,8.1833,-11.1333,36.27,18,74,1.11,SL,1706545040,NaN
70,70,aswan,24.0934,32.9070,18.31,29,9,4.63,EG,1706545250,NaN
81,81,boali,4.8005,18.1275,36.20,13,42,2.08,CF,1706545251,NaN
91,91,bardai,11.6500,7.2333,29.72,10,4,5.49,NG,1706545252,NaN
93,93,alice springs,-23.7000,133.8833,28.45,13,71,0.89,AU,1706545120,NaN
107,107,gidam,18.9833,81.4000,18.90,37,35,0.26,IN,1706545254,NaN
112,112,hermosillo,29.0667,-110.9667,22.14,34,40,1.34,MX,1706545250,NaN
132,132,timbuktu,16.7735,-3.0074,30.25,8,1,3.71,ML,1706545257,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 25000  # 25000 finds fewer hotels but finds enough

params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kandi - nearest hotel: Aéroport de Kandi
bam - nearest hotel: فرودگاه بین المللی بم
panguma - nearest hotel: No hotel found
aswan - nearest hotel: مطار اسوان الدولى
boali - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
alice springs - nearest hotel: Alice Springs Airport
gidam - nearest hotel: No hotel found
hermosillo - nearest hotel: No hotel found
timbuktu - nearest hotel: Aéroport de Tombouctu
chonchi - nearest hotel: Aeródromo Los Calafates
hoshangabad - nearest hotel: No hotel found
bawku - nearest hotel: No hotel found
broken hill - nearest hotel: Broken Hill Airport
sann - nearest hotel: No hotel found
queenstown - nearest hotel: Queenstown Airport
guanajuato - nearest hotel: Del Bajío International Airport
qamdo - nearest hotel: No hotel found
at taj - nearest hotel: مطار الكفرة الدولي
ciudad bolivar - nearest hotel: Aeropuerto Nacional Tomás de Heres
bredasdorp - nearest hotel: AFB Overberg
alaghsas - nearest hotel: Mano Dayak Inte

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,51,kandi,11.1342,2.9386,34.81,10,50,2.12,BJ,1706545247,Aéroport de Kandi
53,53,bam,29.1060,58.3570,13.37,28,100,2.20,IR,1706545247,فرودگاه بین المللی بم
68,68,panguma,8.1833,-11.1333,36.27,18,74,1.11,SL,1706545040,No hotel found
70,70,aswan,24.0934,32.9070,18.31,29,9,4.63,EG,1706545250,مطار اسوان الدولى
81,81,boali,4.8005,18.1275,36.20,13,42,2.08,CF,1706545251,No hotel found
91,91,bardai,11.6500,7.2333,29.72,10,4,5.49,NG,1706545252,No hotel found
93,93,alice springs,-23.7000,133.8833,28.45,13,71,0.89,AU,1706545120,Alice Springs Airport
107,107,gidam,18.9833,81.4000,18.90,37,35,0.26,IN,1706545254,No hotel found
112,112,hermosillo,29.0667,-110.9667,22.14,34,40,1.34,MX,1706545250,No hotel found
132,132,timbuktu,16.7735,-3.0074,30.25,8,1,3.71,ML,1706545257,Aéroport de Tombouctu


In [7]:
# I decided to drop the rows with "No hotel found" just to make the next map more legible and more useful
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df[hotel_df['Hotel Name']!= 'No hotel found']
hotel_df.count()


City_ID       26
City          26
Lat           26
Lng           26
Max Temp      26
Humidity      26
Cloudiness    26
Wind Speed    26
Country       26
Date          26
Hotel Name    26
dtype: int64

In [8]:
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,51,kandi,11.1342,2.9386,34.81,10,50,2.12,BJ,1706545247,Aéroport de Kandi
53,53,bam,29.1060,58.3570,13.37,28,100,2.20,IR,1706545247,فرودگاه بین المللی بم
70,70,aswan,24.0934,32.9070,18.31,29,9,4.63,EG,1706545250,مطار اسوان الدولى
93,93,alice springs,-23.7000,133.8833,28.45,13,71,0.89,AU,1706545120,Alice Springs Airport
132,132,timbuktu,16.7735,-3.0074,30.25,8,1,3.71,ML,1706545257,Aéroport de Tombouctu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Humidity", "Country", "Hotel Name"]
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)